# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup

import pandas as pd
import numpy as np
import requests
import gmaps
import os
import matplotlib.pyplot as plt
import json

# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
# Create aiport dataframe
Cities_df = pd.read_csv('../output_data/CitiesOutput.csv')
Cities_df.dropna(inplace = True) 
Cities_df.head()


,Unnamed: 0,city,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ostrovnoy,68.0531,39.5131,9.80,68,88,8.85,RU,1627219717
1,1,mahebourg,-20.4081,57.7000,23.16,64,40,5.14,MU,1627219886
2,2,qaanaaq,77.4840,-69.3632,4.13,70,77,3.22,GL,1627219887
3,3,zhuhai,22.2769,113.5678,31.09,84,20,4.63,CN,1627219736
4,4,cape town,-33.9258,18.4232,23.23,12,0,2.57,ZA,1627219601


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
# Store latitude and longitude in locations
locations = Cities_df[["lat", "Lng"]]

In [ ]:
Humidity = Cities_df['Humidity'].astype(float)

fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

NameError: name 'Figure' is not defined

In [12]:

hotel_df = Cities_df.loc[((Cities_df["Max Temp"] > ((70-32)*(5/9))) & (Cities_df["Max Temp"] < ((80-32)*(5/9)))) & (Cities_df["Wind Speed"] < (10*1.61)) & (Cities_df["Cloudiness"] == 0)]
hotel_df

,Unnamed: 0,city,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,cape town,-33.9258,18.4232,23.23,12,0,2.57,ZA,1627219601
47,47,port elizabeth,-33.9180,25.5701,23.16,27,0,1.54,ZA,1627219920
117,117,east london,-33.0153,27.9116,21.52,43,0,3.60,ZA,1627219973
161,161,sao joao da barra,-21.6403,-41.0511,24.05,62,0,5.72,BR,1627220006
198,198,grand baie,-20.0182,57.5802,23.26,64,0,3.58,MU,1627220034
252,252,kajaani,64.2273,27.7285,22.03,49,0,5.14,FI,1627220074
274,274,goderich,43.7501,-81.7165,22.89,83,0,2.06,CA,1627220090
281,281,hasaki,35.7333,140.8333,25.22,80,0,5.62,JP,1627220095
298,298,mikkeli,61.6886,27.2723,24.06,46,0,2.06,FI,1627220109
385,385,santa maria,-29.6842,-53.8069,25.61,36,0,9.26,BR,1627219949


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df["Hotel Name"] = ""

target_rank = "distance"
target_radius = 5000
target_type  = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": target_radius,
    "types": target_type,
    "keyword": "hotel",
    "key": g_key
}

# Loop through the cities_df and run a lat/long search for each city to find the first hotel
for index, row in hotel_df.iterrows():

    #get coordinates
    City = row['city']
    # add Coordinates to Parameters
    Lat= row['lat']
    Long = row['Lng']
    Combined = f"{Lat},{Long}"
    params["location"] = Combined

    print(f"Retreiving results for index {index}: {City} - {Combined}")
    response = requests.get(base_url, params=params).json()
    

    try:
        hotel = response["results"][0]['name']
        print(f"Closest hotel is {hotel}.")
        hotel_df.loc[index, 'Hotel Name'] = response["results"][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result....Skipping")
    
    print("-------------")







C:\Users\AjgOI\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Retreiving results for index 4: cape town - -33.9258,18.4232
Closest hotel is Atlanticview Cape Town Boutique Hotel.
-------------
Retreiving results for index 47: port elizabeth - -33.918,25.5701


C:\Users\AjgOI\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Island Way Villa.
-------------
Retreiving results for index 117: east london - -33.0153,27.9116
Closest hotel is Southern Sun Hemingways.
-------------
Retreiving results for index 161: sao joao da barra - -21.6403,-41.0511
Closest hotel is Pousada Porto Prime.
-------------
Retreiving results for index 198: grand baie - -20.0182,57.5802
Closest hotel is Royal Palm Beachcomber Luxury.
-------------
Retreiving results for index 252: kajaani - 64.2273,27.7285
Closest hotel is Scandic Kajanus.
-------------
Retreiving results for index 274: goderich - 43.7501,-81.7165
Closest hotel is Samuels Hotel.
-------------
Retreiving results for index 281: hasaki - 35.7333,140.8333
Closest hotel is Hotel Sunrise Choshi.
-------------
Retreiving results for index 298: mikkeli - 61.6886,27.2723
Closest hotel is Scandic Mikkeli.
-------------
Retreiving results for index 385: santa maria - -29.6842,-53.8069
Closest hotel is Itaimbé Palace Hotel.
-------------
Retreiving results for i

In [14]:
hotel_df.rename(columns={"lat":"Lat"})
hotel_df

,Unnamed: 0,city,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,cape town,-33.9258,18.4232,23.23,12,0,2.57,ZA,1627219601,Atlanticview Cape Town Boutique Hotel
47,47,port elizabeth,-33.9180,25.5701,23.16,27,0,1.54,ZA,1627219920,Island Way Villa
117,117,east london,-33.0153,27.9116,21.52,43,0,3.60,ZA,1627219973,Southern Sun Hemingways
161,161,sao joao da barra,-21.6403,-41.0511,24.05,62,0,5.72,BR,1627220006,Pousada Porto Prime
198,198,grand baie,-20.0182,57.5802,23.26,64,0,3.58,MU,1627220034,Royal Palm Beachcomber Luxury
252,252,kajaani,64.2273,27.7285,22.03,49,0,5.14,FI,1627220074,Scandic Kajanus
274,274,goderich,43.7501,-81.7165,22.89,83,0,2.06,CA,1627220090,Samuels Hotel
281,281,hasaki,35.7333,140.8333,25.22,80,0,5.62,JP,1627220095,Hotel Sunrise Choshi
298,298,mikkeli,61.6886,27.2723,24.06,46,0,2.06,FI,1627220109,Scandic Mikkeli
385,385,santa maria,-29.6842,-53.8069,25.61,36,0,9.26,BR,1627219949,Itaimbé Palace Hotel


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
Humidity = hotel_df['Humidity'].astype(float)

fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))